In [46]:
import skimage.io
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import cv2
from skimage import exposure, color
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

In [26]:
def calcular_promedio_color(imagen):

    promedio_color = imagen.mean(axis=(0, 1))

    data = {
        'Canal_Rojo': promedio_color[0],
        'Canal_Verde': promedio_color[1],
        'Canal_Azul': promedio_color[2]
    }

    df = pd.DataFrame(data, index=[0])

    return df

In [27]:
def media_imagen_threshold(imagen):
    img_gray = color.rgb2gray(imagen)
    img_ampliacion_rango = exposure.rescale_intensity(img_gray, in_range='image', out_range=(0, 1))
    smoothed_image = cv2.GaussianBlur(img_ampliacion_rango, (5, 5), 0)
    imagen_umbralizada_globulos_blancos = (smoothed_image > 0.3).astype(np.uint8) * 255
    imagen_umbralizada_globulos_negros = 255 - imagen_umbralizada_globulos_blancos
    media_negro = imagen_umbralizada_globulos_negros.mean()
    return media_negro

In [28]:
carpeta = 'data'
resultados = []
dfs_res = []

for carpetita in os.listdir(carpeta):
    ruta_carpetita = os.path.join(carpeta,carpetita)

    for nombre_archivo in os.listdir(ruta_carpetita):
        ruta_imagen = os.path.join(ruta_carpetita, nombre_archivo)

        if os.path.isfile(ruta_imagen) and nombre_archivo.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            imagen = skimage.io.imread(ruta_imagen)
            df_resultado = calcular_promedio_color(imagen)
            media_binario = media_imagen_threshold(imagen) 
            df_resultado['media_binario'] = media_binario
            resultados.append(df_resultado)

    df_res = pd.concat(resultados, ignore_index=True)

    if carpetita == 'healthy':
        df_res['label'] = 0
    else:
        df_res['label'] = 1

    dfs_res.append(df_res)

df = pd.concat([dfs_res[0], dfs_res[1]], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)

In [32]:
df

,Canal_Rojo,Canal_Verde,Canal_Azul,media_binario,label
0,145.394605,131.261889,118.443892,1.638547,1
1,146.639702,134.644592,143.854252,19.376492,1
2,153.740083,146.794506,141.727608,0.790580,0
3,144.710439,132.726522,140.780015,17.399657,1
4,115.505912,102.346160,91.972191,1.198673,0
...,...,...,...,...,...
162,158.391714,143.295187,132.170944,0.515177,0
163,144.229062,139.286138,141.463789,3.432459,1
164,167.129062,156.358241,147.125746,0.564468,1
165,144.753259,130.597388,118.966603,9.503654,1


In [52]:
X = df.drop('label', axis=1)
y = df['label']

X_train_total, X_test, y_train_total, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_total, y_train_total, test_size=0.2, random_state=42)

clasificador = LogisticRegression()

clasificador.fit(X_train, y_train)
pred = clasificador.predict(X_val)

accuracy = accuracy_score(y_val, pred)
print(f"Precisión del modelo: {accuracy:.4f}")

Precisión del modelo: 0.7037
